# DDSM Mask RCNN training


In [1]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
import time
import concurrent.futures
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import imgaug
from imgaug import augmenters as iaa

# Root directory of the project
ROOT_DIR = os.getcwd()
print(ROOT_DIR)
# if ROOT_DIR.endswith("samples/nucleus"):
if ROOT_DIR.endswith("mammography"):
    # Go up one level to the repo root
    ROOT_DIR = os.path.dirname(ROOT_DIR)
    print(ROOT_DIR)
    
# Import Mask RCNN
sys.path.append(ROOT_DIR)
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn import model as modellib
from mrcnn.model import log

import mammo

%matplotlib inline 

C:\Users\Chevy\Documents\GitHub\Mammo_MaskRCNN\mammography
C:\Users\Chevy\Documents\GitHub\Mammo_MaskRCNN


C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Comment out to reload imported modules if they change
%load_ext autoreload
%autoreload 2

## Configurations

In [3]:
# Dataset directory
# DATASET_DIR = os.path.join(ROOT_DIR, "datasets/nucleus")
DATASET_DIR = os.path.join(ROOT_DIR, "datasets/mammo")



In [4]:
# Load dataset
dataset_train = mammo.MammoDataset()
dataset_train.load_mammo(DATASET_DIR, subset="train")

# Must call before using the dataset
dataset_train.prepare()

print("Image Count: {}".format(len(dataset_train.image_ids)))
print("Class Count: {}".format(dataset_train.num_classes))
for i, info in enumerate(dataset_train.class_info):
    print("{:3}. {:50}".format(i, info['name']))
    

# Load validation dataset
dataset_val = mammo.MammoDataset()
dataset_val.load_mammo(DATASET_DIR, "val")
dataset_val.prepare()

print("Images: {}\nClasses: {}".format(len(dataset_val.image_ids), dataset_val.class_names))




train
Image Count: 1878
Class Count: 2
  0. BG                                                
  1. mass                                              
val
Images: 476
Classes: ['BG', 'mass']


## Notebook Preferences

In [5]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [6]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=80, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



bn4i_branch2b          (BatchNorm)
res4i_branch2c         (Conv2D)
bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Co

1402/1402 [==============================] - 1184s 844ms/step - loss: 1.1526 - rpn_class_loss: 0.0073 - rpn_bbox_loss: 0.2501 - mrcnn_class_loss: 0.1238 - mrcnn_bbox_loss: 0.3428 - mrcnn_mask_loss: 0.4287 - val_loss: 1.4214 - val_rpn_class_loss: 0.0078 - val_rpn_bbox_loss: 0.4279 - val_mrcnn_class_loss: 0.1332 - val_mrcnn_bbox_loss: 0.4174 - val_mrcnn_mask_loss: 0.4351
Epoch 11/80
1402/1402 [==============================] - 1182s 843ms/step - loss: 1.1304 - rpn_class_loss: 0.0069 - rpn_bbox_loss: 0.2395 - mrcnn_class_loss: 0.1325 - mrcnn_bbox_loss: 0.3290 - mrcnn_mask_loss: 0.4224 - val_loss: 1.4952 - val_rpn_class_loss: 0.0075 - val_rpn_bbox_loss: 0.5301 - val_mrcnn_class_loss: 0.1264 - val_mrcnn_bbox_loss: 0.4047 - val_mrcnn_mask_loss: 0.4266
Epoch 12/80
1402/1402 [==============================] - 1179s 841ms/step - loss: 1.1384 - rpn_class_loss: 0.0066 - rpn_bbox_loss: 0.2476 - mrcnn_class_loss: 0.1408 - mrcnn_bbox_loss: 0.3235 - mrcnn_mask_loss: 0.4199 - val_loss: 1.4591 - val_rp

1402/1402 [==============================] - 1217s 868ms/step - loss: 0.5654 - rpn_class_loss: 0.0039 - rpn_bbox_loss: 0.0633 - mrcnn_class_loss: 0.0839 - mrcnn_bbox_loss: 0.1102 - mrcnn_mask_loss: 0.3041 - val_loss: 1.9384 - val_rpn_class_loss: 0.0087 - val_rpn_bbox_loss: 0.6428 - val_mrcnn_class_loss: 0.2934 - val_mrcnn_bbox_loss: 0.4662 - val_mrcnn_mask_loss: 0.5274
Epoch 32/80
1402/1402 [==============================] - 1215s 867ms/step - loss: 0.5547 - rpn_class_loss: 0.0039 - rpn_bbox_loss: 0.0593 - mrcnn_class_loss: 0.0779 - mrcnn_bbox_loss: 0.1109 - mrcnn_mask_loss: 0.3026 - val_loss: 1.9153 - val_rpn_class_loss: 0.0089 - val_rpn_bbox_loss: 0.5752 - val_mrcnn_class_loss: 0.3209 - val_mrcnn_bbox_loss: 0.4531 - val_mrcnn_mask_loss: 0.5572
Epoch 33/80
1402/1402 [==============================] - 1216s 867ms/step - loss: 0.5249 - rpn_class_loss: 0.0036 - rpn_bbox_loss: 0.0565 - mrcnn_class_loss: 0.0746 - mrcnn_bbox_loss: 0.1010 - mrcnn_mask_loss: 0.2893 - val_loss: 2.1208 - val_rp

1402/1402 [==============================] - 1181s 842ms/step - loss: 0.3273 - rpn_class_loss: 0.0022 - rpn_bbox_loss: 0.0301 - mrcnn_class_loss: 0.0359 - mrcnn_bbox_loss: 0.0520 - mrcnn_mask_loss: 0.2071 - val_loss: 2.6610 - val_rpn_class_loss: 0.0125 - val_rpn_bbox_loss: 0.6657 - val_mrcnn_class_loss: 0.6758 - val_mrcnn_bbox_loss: 0.4670 - val_mrcnn_mask_loss: 0.8400
Epoch 53/80
1402/1402 [==============================] - 1202s 857ms/step - loss: 0.3310 - rpn_class_loss: 0.0021 - rpn_bbox_loss: 0.0319 - mrcnn_class_loss: 0.0367 - mrcnn_bbox_loss: 0.0522 - mrcnn_mask_loss: 0.2081 - val_loss: 2.3266 - val_rpn_class_loss: 0.0113 - val_rpn_bbox_loss: 0.6081 - val_mrcnn_class_loss: 0.6047 - val_mrcnn_bbox_loss: 0.4648 - val_mrcnn_mask_loss: 0.6377
Epoch 54/80
1402/1402 [==============================] - 1200s 856ms/step - loss: 0.3219 - rpn_class_loss: 0.0018 - rpn_bbox_loss: 0.0306 - mrcnn_class_loss: 0.0360 - mrcnn_bbox_loss: 0.0508 - mrcnn_mask_loss: 0.2025 - val_loss: 2.4484 - val_rp

1402/1402 [==============================] - 1205s 859ms/step - loss: 0.2264 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0210 - mrcnn_class_loss: 0.0225 - mrcnn_bbox_loss: 0.0305 - mrcnn_mask_loss: 0.1512 - val_loss: 2.7970 - val_rpn_class_loss: 0.0154 - val_rpn_bbox_loss: 0.6355 - val_mrcnn_class_loss: 0.6779 - val_mrcnn_bbox_loss: 0.4563 - val_mrcnn_mask_loss: 1.0118
Epoch 74/80
1402/1402 [==============================] - 1202s 857ms/step - loss: 0.2260 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0207 - mrcnn_class_loss: 0.0234 - mrcnn_bbox_loss: 0.0296 - mrcnn_mask_loss: 0.1512 - val_loss: 2.8190 - val_rpn_class_loss: 0.0164 - val_rpn_bbox_loss: 0.6178 - val_mrcnn_class_loss: 0.8066 - val_mrcnn_bbox_loss: 0.4472 - val_mrcnn_mask_loss: 0.9309
Epoch 75/80
1402/1402 [==============================] - 1202s 857ms/step - loss: 0.2288 - rpn_class_loss: 0.0010 - rpn_bbox_loss: 0.0208 - mrcnn_class_loss: 0.0249 - mrcnn_bbox_loss: 0.0303 - mrcnn_mask_loss: 0.1518 - val_loss: 2.7882 - val_rp

In [7]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + mass

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = (len(dataset_train.image_ids) - len(dataset_val.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=60, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchNorm)
res4n_branch2c         (Con

1402/1402 [==============================] - 1240s 884ms/step - loss: 0.5575 - rpn_class_loss: 0.0027 - rpn_bbox_loss: 0.0624 - mrcnn_class_loss: 0.0800 - mrcnn_bbox_loss: 0.1128 - mrcnn_mask_loss: 0.2995 - val_loss: 1.8726 - val_rpn_class_loss: 0.0050 - val_rpn_bbox_loss: 0.6627 - val_mrcnn_class_loss: 0.2723 - val_mrcnn_bbox_loss: 0.4478 - val_mrcnn_mask_loss: 0.4847
Epoch 32/60
1402/1402 [==============================] - 1238s 883ms/step - loss: 0.5432 - rpn_class_loss: 0.0026 - rpn_bbox_loss: 0.0599 - mrcnn_class_loss: 0.0761 - mrcnn_bbox_loss: 0.1064 - mrcnn_mask_loss: 0.2981 - val_loss: 1.8566 - val_rpn_class_loss: 0.0051 - val_rpn_bbox_loss: 0.6110 - val_mrcnn_class_loss: 0.2753 - val_mrcnn_bbox_loss: 0.4495 - val_mrcnn_mask_loss: 0.5156
Epoch 33/60
1402/1402 [==============================] - 1241s 885ms/step - loss: 0.5213 - rpn_class_loss: 0.0024 - rpn_bbox_loss: 0.0557 - mrcnn_class_loss: 0.0707 - mrcnn_bbox_loss: 0.1038 - mrcnn_mask_loss: 0.2887 - val_loss: 2.0101 - val_rp

## Train #3
### With 2000 POST_NMS_ROIS_TRAINING and 2500 POST_NMS_ROIS_INFERENCE

In [5]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 2000
    POST_NMS_ROIS_INFERENCE = 2500
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=60, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchNorm)
res4n_branch2c         (Conv2D)
bn4n_branch2c          (BatchNorm)
res4o_branch2a         (Con

Epoch 11/60
984/984 [==============================] - 1173s 1s/step - loss: 0.7815 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.1146 - mrcnn_class_loss: 0.1262 - mrcnn_bbox_loss: 0.1936 - mrcnn_mask_loss: 0.3443 - val_loss: 1.1722 - val_rpn_class_loss: 0.0031 - val_rpn_bbox_loss: 0.3903 - val_mrcnn_class_loss: 0.1228 - val_mrcnn_bbox_loss: 0.2902 - val_mrcnn_mask_loss: 0.3658
Epoch 12/60
984/984 [==============================] - 1177s 1s/step - loss: 0.7264 - rpn_class_loss: 0.0026 - rpn_bbox_loss: 0.0962 - mrcnn_class_loss: 0.1249 - mrcnn_bbox_loss: 0.1706 - mrcnn_mask_loss: 0.3321 - val_loss: 1.1936 - val_rpn_class_loss: 0.0027 - val_rpn_bbox_loss: 0.3595 - val_mrcnn_class_loss: 0.1290 - val_mrcnn_bbox_loss: 0.3180 - val_mrcnn_mask_loss: 0.3843
Epoch 13/60
984/984 [==============================] - 1178s 1s/step - loss: 0.6738 - rpn_class_loss: 0.0024 - rpn_bbox_loss: 0.0799 - mrcnn_class_loss: 0.1157 - mrcnn_bbox_loss: 0.1535 - mrcnn_mask_loss: 0.3223 - val_loss: 1.3115 - val_rpn_c

Epoch 33/60
984/984 [==============================] - 1178s 1s/step - loss: 0.2788 - rpn_class_loss: 9.7027e-04 - rpn_bbox_loss: 0.0257 - mrcnn_class_loss: 0.0380 - mrcnn_bbox_loss: 0.0361 - mrcnn_mask_loss: 0.1780 - val_loss: 2.0119 - val_rpn_class_loss: 0.0025 - val_rpn_bbox_loss: 0.3080 - val_mrcnn_class_loss: 0.6276 - val_mrcnn_bbox_loss: 0.3551 - val_mrcnn_mask_loss: 0.7187
Epoch 34/60
984/984 [==============================] - 1176s 1s/step - loss: 0.2730 - rpn_class_loss: 9.1119e-04 - rpn_bbox_loss: 0.0252 - mrcnn_class_loss: 0.0376 - mrcnn_bbox_loss: 0.0357 - mrcnn_mask_loss: 0.1735 - val_loss: 2.1726 - val_rpn_class_loss: 0.0030 - val_rpn_bbox_loss: 0.3216 - val_mrcnn_class_loss: 0.7483 - val_mrcnn_bbox_loss: 0.3666 - val_mrcnn_mask_loss: 0.7331
Epoch 35/60
984/984 [==============================] - 1177s 1s/step - loss: 0.2653 - rpn_class_loss: 9.1526e-04 - rpn_bbox_loss: 0.0260 - mrcnn_class_loss: 0.0347 - mrcnn_bbox_loss: 0.0343 - mrcnn_mask_loss: 0.1694 - val_loss: 2.3798

## Training #2 
### With proper bounding box and 60 epochs

In [6]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=60, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


bn4i_branch2c          (BatchNorm)
res4j_branch2a         (Conv2D)
bn4j_branch2a          (BatchNorm)
res4j_branch2b         (Conv2D)
bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchNorm)
res4n_branch2c         (Con

Epoch 11/60
984/984 [==============================] - 809s 823ms/step - loss: 0.8031 - rpn_class_loss: 0.0030 - rpn_bbox_loss: 0.1218 - mrcnn_class_loss: 0.1195 - mrcnn_bbox_loss: 0.2000 - mrcnn_mask_loss: 0.3588 - val_loss: 1.1603 - val_rpn_class_loss: 0.0030 - val_rpn_bbox_loss: 0.2916 - val_mrcnn_class_loss: 0.1418 - val_mrcnn_bbox_loss: 0.3378 - val_mrcnn_mask_loss: 0.3862
Epoch 12/60
984/984 [==============================] - 810s 824ms/step - loss: 0.7393 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.0982 - mrcnn_class_loss: 0.1174 - mrcnn_bbox_loss: 0.1757 - mrcnn_mask_loss: 0.3452 - val_loss: 1.1646 - val_rpn_class_loss: 0.0029 - val_rpn_bbox_loss: 0.3261 - val_mrcnn_class_loss: 0.1456 - val_mrcnn_bbox_loss: 0.3173 - val_mrcnn_mask_loss: 0.3727
Epoch 13/60
984/984 [==============================] - 807s 820ms/step - loss: 0.7080 - rpn_class_loss: 0.0027 - rpn_bbox_loss: 0.0878 - mrcnn_class_loss: 0.1193 - mrcnn_bbox_loss: 0.1592 - mrcnn_mask_loss: 0.3391 - val_loss: 1.2166 - val

984/984 [==============================] - 831s 845ms/step - loss: 0.1840 - rpn_class_loss: 6.5612e-04 - rpn_bbox_loss: 0.0159 - mrcnn_class_loss: 0.0209 - mrcnn_bbox_loss: 0.0189 - mrcnn_mask_loss: 0.1277 - val_loss: 2.3100 - val_rpn_class_loss: 0.0033 - val_rpn_bbox_loss: 0.3501 - val_mrcnn_class_loss: 0.8233 - val_mrcnn_bbox_loss: 0.3451 - val_mrcnn_mask_loss: 0.7882
Epoch 54/60
984/984 [==============================] - 832s 846ms/step - loss: 0.1850 - rpn_class_loss: 6.8979e-04 - rpn_bbox_loss: 0.0157 - mrcnn_class_loss: 0.0220 - mrcnn_bbox_loss: 0.0192 - mrcnn_mask_loss: 0.1274 - val_loss: 2.1762 - val_rpn_class_loss: 0.0034 - val_rpn_bbox_loss: 0.3517 - val_mrcnn_class_loss: 0.6564 - val_mrcnn_bbox_loss: 0.3617 - val_mrcnn_mask_loss: 0.8030
Epoch 55/60
984/984 [==============================] - 832s 846ms/step - loss: 0.1807 - rpn_class_loss: 6.4021e-04 - rpn_bbox_loss: 0.0145 - mrcnn_class_loss: 0.0232 - mrcnn_bbox_loss: 0.0187 - mrcnn_mask_loss: 0.1236 - val_loss: 2.1490 - val

## Training #1 
### With proper bounding box and 40 epochs

In [6]:
# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo.MammoConfig):
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = model.get_imagenet_weights()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=40, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

C:\Users\Chevy\Anaconda3\envs\nnets\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



bn4j_branch2b          (BatchNorm)
res4j_branch2c         (Conv2D)
bn4j_branch2c          (BatchNorm)
res4k_branch2a         (Conv2D)
bn4k_branch2a          (BatchNorm)
res4k_branch2b         (Conv2D)
bn4k_branch2b          (BatchNorm)
res4k_branch2c         (Conv2D)
bn4k_branch2c          (BatchNorm)
res4l_branch2a         (Conv2D)
bn4l_branch2a          (BatchNorm)
res4l_branch2b         (Conv2D)
bn4l_branch2b          (BatchNorm)
res4l_branch2c         (Conv2D)
bn4l_branch2c          (BatchNorm)
res4m_branch2a         (Conv2D)
bn4m_branch2a          (BatchNorm)
res4m_branch2b         (Conv2D)
bn4m_branch2b          (BatchNorm)
res4m_branch2c         (Conv2D)
bn4m_branch2c          (BatchNorm)
res4n_branch2a         (Conv2D)
bn4n_branch2a          (BatchNorm)
res4n_branch2b         (Conv2D)
bn4n_branch2b          (BatchNorm)
res4n_branch2c         (Conv2D)
bn4n_branch2c          (BatchNorm)
res4o_branch2a         (Conv2D)
bn4o_branch2a          (BatchNorm)
res4o_branch2b         (Co

Epoch 11/40
984/984 [==============================] - 1350s 1s/step - loss: 0.6850 - rpn_class_loss: 0.0027 - rpn_bbox_loss: 0.1114 - mrcnn_class_loss: 0.0289 - mrcnn_bbox_loss: 0.1996 - mrcnn_mask_loss: 0.3425 - val_loss: 1.1400 - val_rpn_class_loss: 0.0030 - val_rpn_bbox_loss: 0.3213 - val_mrcnn_class_loss: 0.0514 - val_mrcnn_bbox_loss: 0.3276 - val_mrcnn_mask_loss: 0.4367
Epoch 12/40
984/984 [==============================] - 1358s 1s/step - loss: 0.6486 - rpn_class_loss: 0.0024 - rpn_bbox_loss: 0.0993 - mrcnn_class_loss: 0.0269 - mrcnn_bbox_loss: 0.1840 - mrcnn_mask_loss: 0.3360 - val_loss: 1.0837 - val_rpn_class_loss: 0.0025 - val_rpn_bbox_loss: 0.4048 - val_mrcnn_class_loss: 0.0297 - val_mrcnn_bbox_loss: 0.2958 - val_mrcnn_mask_loss: 0.3509
Epoch 13/40
984/984 [==============================] - 1358s 1s/step - loss: 0.5991 - rpn_class_loss: 0.0023 - rpn_bbox_loss: 0.0865 - mrcnn_class_loss: 0.0244 - mrcnn_bbox_loss: 0.1646 - mrcnn_mask_loss: 0.3212 - val_loss: 1.0945 - val_rpn_c

Epoch 33/40
984/984 [==============================] - 1354s 1s/step - loss: 0.2596 - rpn_class_loss: 0.0010 - rpn_bbox_loss: 0.0281 - mrcnn_class_loss: 0.0066 - mrcnn_bbox_loss: 0.0409 - mrcnn_mask_loss: 0.1829 - val_loss: 1.3726 - val_rpn_class_loss: 0.0023 - val_rpn_bbox_loss: 0.3322 - val_mrcnn_class_loss: 0.0812 - val_mrcnn_bbox_loss: 0.3456 - val_mrcnn_mask_loss: 0.6113
Epoch 34/40
984/984 [==============================] - 1361s 1s/step - loss: 0.2577 - rpn_class_loss: 0.0010 - rpn_bbox_loss: 0.0281 - mrcnn_class_loss: 0.0067 - mrcnn_bbox_loss: 0.0406 - mrcnn_mask_loss: 0.1811 - val_loss: 1.7583 - val_rpn_class_loss: 0.0031 - val_rpn_bbox_loss: 0.3568 - val_mrcnn_class_loss: 0.1285 - val_mrcnn_bbox_loss: 0.3535 - val_mrcnn_mask_loss: 0.9164
Epoch 35/40
984/984 [==============================] - 1354s 1s/step - loss: 0.2454 - rpn_class_loss: 9.7563e-04 - rpn_bbox_loss: 0.0250 - mrcnn_class_loss: 0.0055 - mrcnn_bbox_loss: 0.0370 - mrcnn_mask_loss: 0.1769 - val_loss: 1.5053 - val_r